## Getting Started with PointLogic via Python

This notebook uses python to make PointLogic API requests

- Navigate through PointLogic endpoints 
- Pull PointLogic Views and the metadata
- Generate final PointLogic endpoint for Natural Gas Production and load into a DataFrame

Use cases:
- How to pull multiple years of Volume History

###### created on 19 Dec 2023 
###### updated on 01 May 2024
###### version 0.6
###### Disclaimer: The sample code provided is for informational purposes only and should not be considered as professional advice. It is important to thoroughly review and test the code before implementing it in a production environment. S&P Global does not guarantee the accuracy, reliability, or completeness of the code and shall not be held liable for any damages or losses arising from its use.

## Initial Setup

In [1]:
# uncomment below to install dependencies
# !pip install requests
# !pip install pandas
# !pip install numpy
# !pip install time

In [1]:
import requests
from requests.auth import HTTPBasicAuth
import pandas as pd
import numpy as np
import time

## Generate your personal access token

In the ConnectAPI Landing Page, click on the Personal Access Tokens tile. There you'll find all your active tokens. Click on New Token, give a name to this token, check the "Never expire" box, and click Create.
**A one time pop-up will show your username and password, save these as they won't show up again.**

If you ever want to change your token, you can delete the old one and generate a new one with the prior process.

In [2]:
# Paste your Personal Access Token username and password inside the quotes
username = ''
password = ''

# PointLogic 

In the ConnectAPI Landing Page, click on the Service Documentation (Swagger) tile. PointLogic can be found under "Climate and Sustainability", PointLogic and PointLogic Views. 

### PointLogic lookup____

Within PointLogic, there are 10 endpoints call `lookup____` which return the codes that work within the parameters. For example, lookup_region returns the codes for regions in Canada, USA, and Mexico.

In [3]:
params = {
    "pageSize": 10000,
    "pageIndex": 1
  }
r = requests.get("https://api.connect.ihsmarkit.com/cs/v1/pointlogic/lookup_region", auth=HTTPBasicAuth(username, password), params=params)
#print(r)
r.raise_for_status()
j = r.json()
df1= pd.DataFrame.from_dict(j)
df1

,name,id
0,Canada,26158
1,Mexico,26160
2,Mid-Continent,26105
3,Northeast,26104
4,Rocky Mountain,26106
5,Southeast,26107
6,Texas,26108
7,Western,26109


Some `lookup____` calls require other `lookup____` codes to get a response, such as lookup_subregions needs a code from lookup_region

In [5]:
params = {
    "pageSize": 10000,
    "pageIndex": 1
  }
r = requests.get("https://api.connect.ihsmarkit.com/cs/v1/pointlogic/lookup_subregion/26104", auth=HTTPBasicAuth(username, password), params=params)
#print(r)
r.raise_for_status()
j = r.json()
df2= pd.DataFrame.from_dict(j)
df2

,id,name
0,26249,Atlantic
1,26250,Appalachia
2,26255,New England
3,26256,NY/NJ


### Retrieve PointLogic Data

Natural Gas Production can be found in `pointlogic/production/naturalGas` and using the paramaters found, generate a url to pull the data you'd like. 
- `startDate` uses the format (MM-DD-YYYYY)
- `pageIndex` chooses which page (if multiple) you'd like to pull. If left blank, all pages will be pulled 
- `pageSize` chooses how many records per page, with a default and limit of 10,000

# PointLogic Views

PointLogic Views shows the metadata of the datasets within PointLogic

Beginning at `plviews/views`, this returns a list of all the datasets

In [7]:
params = {
    "pageSize": 10000,
    "pageIndex": 1
  }
r = requests.get("https://api.connect.ihsmarkit.com/cs/v1/plview/views", auth=HTTPBasicAuth(username, password), params=params)
#print(r)
r.raise_for_status()
j = r.json()
df3= pd.DataFrame(j)
df3

,0
0,aggregatesampleusdemand_pipe_region_state_type
1,demandforecast_region
2,demandforecast_uslower48
3,flowsby_lciid
4,flowsby_points_zone
5,gasproduction_point
6,gasproduction_producingarea
7,lookup_pipeline
8,lookup_pipelinenoticecategory
9,marketbalances_regions


## PL/views/{viewName}
Using `plview/views/{viewName}`, you can see the metadata and definitions of the parameters for each dataset

In [6]:
params = {
    "pageSize": 10000,
    "pageIndex": 1
  }
r = requests.get("https://api.connect.ihsmarkit.com/cs/v1/plview/views/gasproduction_producingarea", auth=HTTPBasicAuth(username, password), params=params)
#print(r)
r.raise_for_status()
j = r.json()
print(j["description"])
df4= pd.DataFrame(j["elements"])
df4

A rolling 31 day accounting of modeled natural gas production at a wet gas (wellhead) value and dry gas value (pipeline grade). The modeled production is aggregated by producing area and their respective state and region. Volumes are expressed as million cubic feet per day (MMcf/d).


,short_name,long_name,description,column_type,column_size,column_decimals,column_is_primary_key,column_is_nullable
0,startdate,startdate,,text,65536,0,False,True
1,regiontypeid,regiontypeid,,text,65536,0,False,True
2,enddate,enddate,,text,65536,0,False,True
3,reporteddate,reporteddate,Date of query execution,text,16,0,False,True
4,referencedate,referencedate,Calendar date that is synomous with the define...,text,16,0,False,True
5,region,region,PointLogic defined region name. Regions are a ...,text,100,0,False,True
6,producingarea,producingarea,PointLogic definition for a geographic collect...,text,100,0,False,True
7,state,state,PointLogic defined state name. States are a co...,text,2,0,False,True
8,dryfactoredvalue,dryfactoredvalue,Natural gas production volume in MMcf/d that r...,decimal,38,8,False,True
9,wellheadvalue,wellheadvalue,Natural gas production volume in MMcf/d prior ...,decimal,38,6,False,True


## PLView/count/{viewName}
`plview/count/{viewName}`returns a count of all the records in the dataset

In [9]:
params = {
    "pageSize": 10000,
    "pageIndex": 1
  }
r = requests.get("https://api.connect.ihsmarkit.com/cs/v1/plview/count/gasproduction_producingarea", auth=HTTPBasicAuth(username, password), params=params)
#print(r)
r.raise_for_status()
j = r.json()
print(j)

2852


Some responses don't include headers, for those use this script to pull the data into a dataframe

In [3]:
r = requests.get("https://api.connect.ihsmarkit.com/cs/v1/plview/retrieve/flowsby_points_zone", auth=HTTPBasicAuth(username, password))
print(r)
r.raise_for_status()
j = r.json()
#print(j[0])
df4= pd.DataFrame(j)
df4

<Response [200]>


,reporteddate,flowdate,pointlciid,volume,operationalcapacity,pointzone
0,2024-03-20 15:24:52,2024-03-23,CAGT00001,1675.000000,,
1,2024-03-20 15:24:52,2024-03-23,CAGT00002,163.000000,,
2,2024-03-20 15:24:52,2024-03-23,CAGT00003,11.000000,,
3,2024-03-20 15:24:52,2024-03-23,CAGT00004,0.000000,,
4,2024-03-20 15:24:52,2024-03-23,CAGT00005,28.000000,,
...,...,...,...,...,...,...
869311,2024-03-20 15:24:52,2023-12-10,STORA0111,-1891.827156,,
869312,2024-03-20 15:24:52,2023-12-10,STORA0114,-227.439901,,
869313,2024-03-20 15:24:52,2023-12-10,STORA0117,-821.041297,,
869314,2024-03-20 15:24:52,2023-12-10,STORA0120,-6356.674401,,


# This function loops through every page in a pull and organizes it into a DataFrame

In [4]:
def get_data(username, password, url):
  params = {
    "pageSize": 10000,
    "pageIndex": 1
  }
  df = pd.DataFrame()
  try:
    while True:
      r = requests.get(url, auth=HTTPBasicAuth(username, password), params=params, verify=True)
      #print(r)
      r.raise_for_status()
      j = r.json()
      #print(j)
      print(f"Page: {params['pageIndex']}")
      if len(j['Data']) == 0:
        break
      params['pageIndex'] += 1
      
      new_df = pd.json_normalize(data=j, record_path="Data")
      df = pd.concat([df, new_df], ignore_index=True)
      time.sleep(1)
  except Exception as err:
    if r.status_code >= 500:
      print (err)
    else:
      print(r.status_code, r.json())
    raise
  
  return df

The following URL as a sample

In [5]:
url = 'https://api.connect.ihsmarkit.com/cs/v1/pointlogic/volumeHistory/pipelineFlows?historyRecords=9&regionId=26104'

Using the username, password, and url created, pull data with the function get_data

In [6]:
df5 = get_data(username, password, url)
df5

Page: 1
Page: 2
Page: 3
Page: 4


,referencedate,pointid,displayname,pipelineid,pipelinedisplayname,pointtypeid,pointtypename,flowdirection,regionid,regionname,subregionid,subregionname,stateid,statename,countyid,countyname,locprop,pointisactive,operatingcapacity,volume
0,2024-06-04,727,North Haven,2,Algonquin Gas Transmission,7,Distribution,Delivery,26104,Northeast,26255,New England,4525,Connecticut,16067,New Haven,00003,1,342.08,-28.354
1,2024-06-04,728,"National Grid - Dey St. (Providence, RI)",2,Algonquin Gas Transmission,7,Distribution,Delivery,26104,Northeast,26255,New England,4557,Rhode Island,16350,Providence,00004,1,157.44,-4.673
2,2024-06-04,738,Stony Point,2,Algonquin Gas Transmission,7,Distribution,Delivery,26104,Northeast,26256,NY/NJ,4550,New York,16460,Rockland,00040,1,235.19,-5.025
3,2024-06-04,739,Peekskill,2,Algonquin Gas Transmission,7,Distribution,Delivery,26104,Northeast,26256,NY/NJ,4550,New York,17030,Westchester,00041,1,67.14,-6.000
4,2024-06-04,748,National Grid - Ponkapoag,2,Algonquin Gas Transmission,7,Distribution,Delivery,26104,Northeast,26255,New England,4539,Massachusetts,16095,Norfolk,00052,1,393.81,-11.707
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24326,2024-06-13,48974,Rel Lateral MP 1.0 W to E NL,214,Transcontinental Pipeline,13,Compressor,Receipt,26104,Northeast,26250,Appalachia,4556,Pennsylvania,15589,Lancaster,9012723,1,453.50,164.977
24327,2024-06-13,48983,TCO Deliveries,75,Equitrans Transmission,5,Interstate,Delivery,26104,Northeast,26250,Appalachia,4566,West Virginia,17034,Wetzel,TCO_DEL,1,100.00,-169.932
24328,2024-06-13,49000,Heracles IC,75,Equitrans Transmission,14,Supply Source,Receipt,26104,Northeast,26250,Appalachia,4556,Pennsylvania,14052,Allegheny,M5466401,1,0.00,89.000
24329,2024-06-13,49088,Mainline Mp 1521.82 Tsb N To S Nl,214,Transcontinental Pipeline,13,Compressor,Receipt,26104,Northeast,26249,Atlantic,4564,Virginia,15724,Louisa,9012963,1,2599.00,1682.542


In [17]:
df5.to_csv("test.csv", index=False)

# Special Usecases
## Large Data Pulls
The PointLogic API tends to not like it when calls of pages 20+ are made. In order to circumvent that limit, specific codes for these scenarios are made.

### Multi-year historical Pipelineflow data
In order to pull historical data that span more than a year, here's a sample script on how to make that data pull using VolumeHistory, credentials, Pipelines, and different PointTypes (Throughputs, Processing, Storage, etc).

In [ ]:
# List of pipelines that will be searched
# pipeline_id= [2,64,90,205]
pipeline_id =[609]
# PointId 
id= {}
# StatDate
startDate= "2018-01-01"

This section will loop through every pipelines chosen to pull out the counties thos pipelines cross through. Then using the counties, the next loop will pull the data county by county to ensure there isn't a time out from our API.
-  In the URL for county_data, the startDate can be adjusted to match your desired date
-  If you'd like to pull a specific pointTypeId, replace county_data URL with the following sample with poinTypeId on whichever number you want f"https://api.connect.ihsmarkit.com/cs/v1/pointlogic/volumeHistory/pipelineFlows?startDate={startDate}&pipelineId={id}&pointTypeId=11&countyId={county}"


In [ ]:
df= pd.DataFrame()

#Loop through every pipeline
for id in pipeline_id:
    get_county = get_data(username, password, f"https://api.connect.ihsmarkit.com/cs/v1/pointlogic/volumeHistory/pipelineFlows?historyRecords=7&pipelineId={id}")
    
    #Turn countyid into an array to remove NA
    test = get_county['countyid'].unique()
    test = test[~np.isnan(test)].astype('int64')
    
    #Loop through each county for the pipeline to pull the data into a dataframe 
    for county in test:
        if pd.isna(county):
            continue
        county_data= get_data(username, password, f"https://api.connect.ihsmarkit.com/cs/v1/pointlogic/volumeHistory/pipelineFlows?startDate={startDate}&pipelineId={id}&countyId={county}")
        df = pd.concat([df, county_data])
df.head()

### Pull Natural Gas Production
If you'd like to pull 20+ pages of data, use this sample script that will allow the API to return every page.

In [ ]:
# RegionsId can be replaced with SubregionID or StateID, or add any other parameter to the URL
regions= []
startDate= ""
df= pd.DataFrame()

# Idea, use the PLviews for natgas production to be the reference data. Then use a column to break down the large production data
for region in regions:
    region_data= get_data(username, password, "https://api.connect.ihsmarkit.com/cs/v1/pointlogic/production/naturalGas?startDate={startDate}&regionId={region}")
    df = pd.concat([df, region_data])

### Pull Gas Production PLView Historical
To pull older data than PLView/gasproduction_point, use the Pipeline flow endpoint. You can call both pointTypeIds (2 and 14) at once. This can be filtered down to the Region, State, Dates you want.

In [13]:
url= "https://api.connect.ihsmarkit.com/cs/v1/pointlogic/volumeHistory/pipelineFlows?pointTypeId=2%2C14"

get_data(username, password, url)

Page: 1
Page: 2
Page: 3


,referencedate,pointid,displayname,pipelineid,pipelinedisplayname,pointtypeid,pointtypename,flowdirection,regionid,regionname,subregionid,subregionname,stateid,statename,countyid,countyname,locprop,pointisactive,operatingcapacity,volume
0,2024-05-02,1309,Beaver Plant (CNG),4,ANR Pipeline,2,Processing,Receipt,26105.0,Mid-Continent,26252.0,Lower Midcon,4554,Oklahoma,14166,Beaver,163450,1,121.0,24.791
1,2024-05-02,1437,Blue Forest,33,Colorado Interstate Gas,2,Processing,Receipt,26106.0,Rocky Mountain,26258.0,Rocky Mountains,4568,Wyoming,16752,Sweetwater,891645,1,149.0,0.170
2,2024-05-02,1498,Echo Springs,33,Colorado Interstate Gas,2,Processing,Receipt,26106.0,Rocky Mountain,26258.0,Rocky Mountains,4568,Wyoming,16752,Sweetwater,800235,1,238.0,90.164
3,2024-05-02,1746,MGR/CIG (LOS) Lost Cabin,33,Colorado Interstate Gas,2,Processing,Receipt,26106.0,Rocky Mountain,26258.0,Rocky Mountains,4568,Wyoming,14997,Fremont,895009,1,142.0,17.468
4,2024-05-02,1755,Mewbourn,33,Colorado Interstate Gas,2,Processing,Receipt,26106.0,Rocky Mountain,26258.0,Rocky Mountains,4524,Colorado,17024,Weld,800421,1,324.0,232.269
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16729,2024-04-25,1689,Frederick,33,Colorado Interstate Gas,14,Supply Source,Receipt,26106.0,Rocky Mountain,26258.0,Rocky Mountains,4524,Colorado,15600,Las Animas,800251,1,23.0,3.600
16730,2024-04-25,1497,East Rock Springs,33,Colorado Interstate Gas,14,Supply Source,Receipt,26106.0,Rocky Mountain,26258.0,Rocky Mountains,4568,Wyoming,16752,Sweetwater,800234,1,11.0,0.550
16731,2024-04-25,1036,Green Station,4,ANR Pipeline,14,Supply Source,Receipt,26105.0,Mid-Continent,26252.0,Lower Midcon,4554,Oklahoma,14171,Beckham,89222,1,40.0,6.600
16732,2024-04-25,1448,Burro Canyon,33,Colorado Interstate Gas,14,Supply Source,Receipt,26106.0,Rocky Mountain,26258.0,Rocky Mountains,4524,Colorado,15600,Las Animas,800107,1,194.0,18.500


To pull data older than PLView/gasproduction_producingarea, use the production/naturalGas endpoint. Using Region, Subregion, State, and startDate as parameters, you'll be able to return the same data. 

In [9]:
url= "https://api.connect.ihsmarkit.com/cs/v1/pointlogic/production/naturalGas"

get_data(username, password, url)

Page: 1
Page: 2
Page: 3


,date,datasettype,regionid,region,subregionid,subregion,stateid,state,producingareaid,producingarea,volume
0,2024-05-01,Modeled Dry,26104,Northeast,26249,Atlantic,4564,Virginia,4288,Other-VA,224.918197
1,2024-05-01,Modeled Wellhead,26104,Northeast,26249,Atlantic,4564,Virginia,4288,Other-VA,213.065372
2,2024-05-01,Modeled Dry,26104,Northeast,26250,Appalachia,4535,Kentucky,4258,Other-KY,183.403263
3,2024-05-01,Modeled Wellhead,26104,Northeast,26250,Appalachia,4535,Kentucky,4258,Other-KY,138.409205
4,2024-05-01,Modeled Dry,26104,Northeast,26250,Appalachia,4553,Ohio,4275,Other-OH,1.356105
...,...,...,...,...,...,...,...,...,...,...,...
13720,2024-03-02,Canadian Modeled Wellhead,26158,Canada,26423,Western Canada,22864,Saskatchewan,24205,Other Southwest-SK,148.085720
13721,2024-03-02,Canadian Modeled Dry,26158,Canada,26423,Western Canada,22864,Saskatchewan,24206,Viking-SK,76.718827
13722,2024-03-02,Canadian Modeled Wellhead,26158,Canada,26423,Western Canada,22864,Saskatchewan,24206,Viking-SK,91.791990
13723,2024-03-02,Modeled Dry,26159,Gulf of Mexico,0,Gulf of Mexico,4529,Gulf of Mexico,4233,Gulf of Mexico,1779.588923


### Pull Market Balance PLView Historical
To pull data older than PLView/marketbalances_regions and marketbalances_uslower48, use the volumeHistory endpoint. Using Region/Subrergion you want retreive, filter with reportDate as parameters, you'll be able to return the same data. 

In [ ]:
url= "https://api.connect.ihsmarkit.com/cs/v1/pointlogic/supplyDemand/region/26265"

get_data(username, password, url)

### Pull Throughput PLView Historical
To pull data older than PLView/pipelineflow_throughputs, use the volumeHistory endpoint. Using pointTypeId= 12, filter with Region, Subregion, State, and startDate as parameters, you'll be able to return the same data. 

In [14]:
url= "https://api.connect.ihsmarkit.com/cs/v1/pointlogic/volumeHistory/pipelineFlows?pointTypeId=12"

get_data(username, password, url)

Page: 1
Page: 2


,referencedate,pointid,displayname,pipelineid,pipelinedisplayname,pointtypeid,pointtypename,flowdirection,regionid,regionname,subregionid,subregionname,stateid,statename,countyid,countyname,locprop,pointisactive,operatingcapacity,volume
0,2024-05-02,712,MS to TN,59,American Midstream Alatenn,12,Throughput,TD1,26107.0,Southeast,26260.0,Southern Corridor,4542,Mississippi,14038.0,Alcorn,,1,18.0,12.92100
1,2024-05-02,713,MS to AL,59,American Midstream Alatenn,12,Throughput,TD1,26107.0,Southeast,26260.0,Southern Corridor,4542,Mississippi,16807.0,Tishomingo,,1,200.0,6.78800
2,2024-05-02,1127,OK to KS East,25,Enable Gas Transmission,12,Throughput,TD1,26105.0,Mid-Continent,26252.0,Lower Midcon,4554,Oklahoma,15482.0,Kay,,1,40.0,3.64800
3,2024-05-02,1128,OK to AR,25,Enable Gas Transmission,12,Throughput,TD1,26105.0,Mid-Continent,26252.0,Lower Midcon,4554,Oklahoma,15622.0,Le Flore,,1,1000.0,1122.00200
4,2024-05-02,1129,AR to LA,25,Enable Gas Transmission,12,Throughput,TD1,26105.0,Mid-Continent,26252.0,Lower Midcon,4522,Arkansas,15560.0,Lafayette,,1,850.0,66.68324
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3843,2024-04-25,1129,AR to LA,25,Enable Gas Transmission,12,Throughput,TD1,26105.0,Mid-Continent,26252.0,Lower Midcon,4522,Arkansas,15560.0,Lafayette,,1,850.0,53.26308
3844,2024-04-25,1128,OK to AR,25,Enable Gas Transmission,12,Throughput,TD1,26105.0,Mid-Continent,26252.0,Lower Midcon,4554,Oklahoma,15622.0,Le Flore,,1,1000.0,1067.71900
3845,2024-04-25,712,MS to TN,59,American Midstream Alatenn,12,Throughput,TD1,26107.0,Southeast,26260.0,Southern Corridor,4542,Mississippi,14038.0,Alcorn,,1,18.0,17.04300
3846,2024-04-25,713,MS to AL,59,American Midstream Alatenn,12,Throughput,TD1,26107.0,Southeast,26260.0,Southern Corridor,4542,Mississippi,16807.0,Tishomingo,,1,200.0,2.81300


### Pull Multi-Year Regional Flows
If you're looking to pull Regional Flows Historical and the total amount of pages is greater than 15, the following script will subdivide the call into regions of the US and then put them together. The startDate can be adjusted in the URL to match what you're looking for.

In [12]:
params = {
    "pageSize": 10000,
    "pageIndex": 1
  }
r = requests.get("https://api.connect.ihsmarkit.com/cs/v1/pointlogic/lookup_region", auth=HTTPBasicAuth(username, password), params=params)
#print(r)
r.raise_for_status()
j = r.json()
df1= pd.DataFrame.from_dict(j)
df2 = df1[2:]

list = df2['id'].tolist()
names = df2['name'].tolist()

In [13]:
df= pd.DataFrame()
for id, name in zip(list, names): 
    print(name)

    url = f"https://api.connect.ihsmarkit.com/cs/v1/pointlogic/regionalFlows/regionSummary?startDate=2014-01-01&regionId={id}"
    data_df= get_data(username, password, url)

    df= pd.concat([df, data_df])

df

Mid-Continent
Page: 1
Page: 2
Page: 3
Page: 4
Page: 5
Page: 6
Page: 7
Northeast
Page: 1
Page: 2
Page: 3
Page: 4
Page: 5
Rocky Mountain
Page: 1
Page: 2
Page: 3
Page: 4
Page: 5
Southeast
Page: 1
Page: 2
Page: 3
Page: 4
Page: 5
Page: 6
Page: 7
Texas
Page: 1
Page: 2
Page: 3
Page: 4
Page: 5
Page: 6
Western
Page: 1
Page: 2
Page: 3
Page: 4
Page: 5
Page: 6


,date,regionid,region_point_of_view,subgrouping,subgroup_ranking,name,volume
0,2014-04-19,26105,Mid-Continent,Northeast,Group 1,Midwest & Appalachia,1270.520996
1,2014-04-19,26105,Mid-Continent,Northeast,Group 1.1,Northeast,1271.000000
2,2014-04-20,26105,Mid-Continent,Northeast,Group 1,Midwest & Appalachia,1307.224992
3,2014-04-20,26105,Mid-Continent,Northeast,Group 1.1,Northeast,1307.000000
4,2014-04-21,26105,Mid-Continent,Northeast,Group 1,Midwest & Appalachia,1173.135008
...,...,...,...,...,...,...,...
45535,2024-05-18,26109,Western,Western,Group 5.2,Net Flows,3390.000000
45536,2024-05-19,26109,Western,Western,Group 5.2,Net Flows,3141.000000
45537,2024-05-20,26109,Western,Western,Group 5.2,Net Flows,3337.000000
45538,2024-05-21,26109,Western,Western,Group 5.2,Net Flows,3265.000000
